# Чат-бот для Telegram

Исходные данные:
1. Словарь намерений BOT_CONFIG на тему еды и некоторые общие темы (т.е. у бота можно, например, спросить: "Что можно быстро приготовить?" или "Что можно приготовить к макаронам? и т.д.)
2. Файл dialogues.txt, представляющий собой сборник диалогов из художественной литературы.

Архитектура чат-бота:
1. Возвращаем заготовленный ответ, извлекаемый из словаря намерений BOT_CONFIG.
2. Используем генеративную модель в том случае, если не находим подходящего намерения.
3. Используем ответ-заглушку, если заготовленный ответ и генеративная модель не дают результата.

Для классификации намерений будем использовать классификаторы LogisticRegression и LinearSVC. LogisticRegression используем для определения вероятности правильной классификации намерения. LinearSVC для нахождения лучшего намерения. Применение двух разных классификаторов связано с тем, что LinearSVC показывает более высокую точность классификации, чем LogisticRegression. Но, в отличие от последнего, не имеет метода predict_proba() (возвращает вероятностные оценки), необходимого для проверки адекватности классикации.

Для генерации ответа будем использовать расстояние Левенштейна для определения наиболее подходящего ответа.

Различные варианты ответов-заглушек будем брать из того же BOT_CONFIG, находящихся там под ключом failure_phrases

In [1]:
import random
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pickle

In [2]:
with open('BOT_CONFIG.pickle', 'rb') as f:
    BOT_CONFIG = pickle.load(f)

Для начала необходимо векторизовать все наши намерения, т.е. преобразовать фразы в некоторые числовые последовательности, которые будут "понятны" классификаторам.
Будем использовать TfidfVectorizer с параметрами analyzer='char' и ngram_range=(2, 3), т.е. будем разбивать фразы на n-граммы по 2 или 3 буквы.

In [3]:
# Подготавливаем данные для векторизации
corpus = []
y = []
for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        corpus.append(example)
        y.append(intent)

# Векторизируем с помощью tf-idf векторайзера 
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
X = vectorizer.fit_transform(corpus)

Как было сказано выше, используем логистическую регрессию для определения вероятности правильной классификации намерения. Будем использовать RandomizedSearchCV для подбора лучших в данном случае параметров классификатора. Также используем кросс-валидацию, которая даст нам более верный набор параметров, т.к. данные из тренировочного датасета будут дополнительно разбиты на n частей (параметр cv=n в RandomizedSearchCV), далее модель будет обучена на n-1 из них, затем протестирована на оставшейся (такое действие будет выполнено для каждой из n частей).

In [4]:
# Разбиваем исходный набор данных на тренировочный и тестовый датасеты
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Задаём параметры и их значения, которые необходимо перебрать
parameters = {'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [0.5, 0.75, 1], 'fit_intercept': [False, True], 'random_state': [42],
             'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}

# Подбираем лучшие параметры
clf_proba = LogisticRegression()
rand_search_clf = RandomizedSearchCV(clf_proba, parameters, cv=5)
rand_search_clf.fit(X_train, y_train)
clf_proba = rand_search_clf.best_estimator_
clf_proba.score(X_test, y_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was r

0.3296398891966759

Точность LogisticRegression составляет 34,1%, что вполне неплохо, учитывая очень низкое качество словаря намерений BOT_CONFIG.

In [5]:
# Обучаем классификатор LogisticRegression на полном наборе данных
clf_proba.fit(X, y)

LogisticRegression(C=0.75, fit_intercept=False, random_state=42, solver='saga',
                   tol=1e-05)

Используем метод опорных вукторов для нахождения лучшего намерения. Здесь также будем использовать RandomizedSearchCV для подбора лучших в данном случае параметров классификатора и кросс-валидацию.

In [6]:
# Разбиваем исходный набор данных на тренировочный и тестовый датасеты
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Задаём параметры и их значения, которые необходимо перебрать
parameters = {'tol': [1e-3, 1e-4, 1e-5, 1e-6], 'C': [0.5, 0.75, 1], 'random_state': [42], 
              'loss': ['hinge', 'squared_hinge'], 'multi_class': ['ovr', 'crammer_singer']}

# Подбираем лучшие параметры
clf = LinearSVC()
rand_search_clf = RandomizedSearchCV(clf, parameters, cv=5)
rand_search_clf.fit(X_train, y_train)
clf = rand_search_clf.best_estimator_
clf.score(X_test, y_test)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\User\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.40166204986149584

Точность LinearSVC составляет 40,2%. Низкая точность опять же объясняется низким качеством словаря намерений BOT_CONFIG.

In [7]:
# Обучаем классификатор LinearSVC на полном наборе данных
clf.fit(X, y)

LinearSVC(C=0.75, random_state=42, tol=1e-05)

Далее приведены две функции для получения заготовленного ответа.

In [8]:
def get_intent(question):
    """Предсказываем лучший intent и проверяем адекватность его классификации 
        по минимальной вероятности (минимальному порогу)"""
    
    # Предсказываем лучшее намерение с помощью LinearSVC
    best_intent = clf.predict(vectorizer.transform([question]))[0]

    # Находим вероятность правильной классификации намерения
    index_of_best_intent = list(clf_proba.classes_).index(best_intent)
    probabilities = clf_proba.predict_proba(vectorizer.transform([question]))[0]
    
    # проверяем адекватность классификации по минимальной вероятности (минимальному порогу)
    min_treshold = 0.05
    best_intent_proba = probabilities[index_of_best_intent]
    if best_intent_proba > min_treshold:
        return best_intent

In [9]:
def get_answer_by_intent(intent):
    """Получаем случайный из возможных ответ на намерение"""
    phrases = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(phrases)

Теперь подготавливаем данные для генеративной модели. Создаём датасет вида {word1: [[question1, answer1], [question2, aanswer2], ...], ...}. После чего удаляем самые популярные слова, т.к. они встречаются почти в каждой фразе, а значит не имеют значения при классификации.

In [10]:
# Подготавливаем данные для генеративной модели
with open('dialogues.txt', encoding='utf8') as f:
    content = f.read()

dialogues = content.split('\n\n')

def clear_question(question):
    """Очистка строки от лишних символов"""
    question = question.lower().strip()
    alphabet = ' -1234567890йцукенгшщзхъфывапролджэёячсмитьбю'
    question = ''.join(c for c in question if c in alphabet)
    return question

# Создание датасета вида {word1: [[q1, a1], [q2, a2], ...], ...}
questions = set()
dataset = {}

for dialogue in dialogues:
    replicas = dialogue.split('\n')[:2]
    if len(replicas) == 2:
        question, answer = replicas
        question = clear_question(question[2:])
        answer = answer[2:]
        
        if question and question not in questions:
            questions.add(question)
            words = question.split(' ')
            for word in words:
                if word not in dataset:
                    dataset[word] = []
                dataset[word].append([question, answer])

# Удаляем самые популярные словы из датасета
too_popular = set()
for word in dataset:
    if len(dataset[word]) > 10000:
        too_popular.add(word)

for word in too_popular:
    dataset.pop(word)

Далее приведена функция для генерирования ответа. Здесь сначала создаём небольшой датасет, только с теми словами, которые входят в фразу, приходящую на вход. Делаем это для того, чтобы не перебирать каждый раз большой датасет (в некоторых случаях на это требуется много времени). После чего возвращаем ответ с минимальным расстоянием Левенштейна (расстояние Левенштейна находим с помощью функции edit_distance() библиотеки nltk.

In [11]:
def get_generative_answer(replica):
    """Получаем ответ, основанный на генеративной модели"""
    
    # Очищаем входную фразу от лишних символов
    replica = clear_question(replica)
    words = replica.split(' ')
    
    # Формируем небольшой датасет
    mini_dataset = []
    for word in words:
        if word in dataset:
            mini_dataset += dataset[word]

    # Находим минимальное расстояние Левенштейна
    candidates = []
    for question, answer in mini_dataset:
        if abs(len(question) - len(replica)) / len(question) < 0.4:
            d = nltk.edit_distance(question, replica)
            diff = d / len(question)
            if diff < 0.4:
                candidates.append([question, answer, diff])
    
    if candidates: 
        winner = min(candidates, key=lambda candidate: candidate[2])
        return winner[1]

Далее приведена функция для получения ответа-заглушки.

In [12]:
def get_failure_phrase():
    """Получаем случайный ответ-заглушку"""
    phrases = BOT_CONFIG['failure_phrases']
    return random.choice(phrases)

Далее приведена основная функция бота.

In [13]:
def bot(question):
    #
    # NLU
    intent = get_intent(question)

    #
    # Получение ответа

    # Заготовленный ответ
    if intent:
        return get_answer_by_intent(intent)

    # Применяем генеративную модель
    answer = get_generative_answer(question)
    if answer:
        return answer

    # Ответ-заглушка
    return get_failure_phrase()

Далее приведен код простейшего Telegram бота. 

In [ ]:
# ! pip install python-telegram-bot

In [ ]:
CONFIG = {'token': '!!!Insert your Telegram bot token!!!'}

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update, context):
    """Echo the user message."""
    question = update.message.text
    answer = bot(question)
    update.message.reply_text(answer)


def main():
    """Start the bot."""
    updater = Updater(CONFIG['token'], use_context=True)

    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    updater.start_polling()
    updater.idle()

In [ ]:
main()